In [1]:
%matplotlib inline


Palm Plantation Prediction : WiDS 2019
=============================

**Author:** Prajna Prabhakara




**Reference**
Using Transfer Learning :
Finetuning and Feature extraction from the `torchvision
models <https://pytorch.org/docs/stable/torchvision/models.html>`

In **feature extraction**,
we start with a pretrained model and only update the final layer weights
from which we derive predictions. It is called feature extraction
because we use the pretrained CNN as a fixed feature-extractor, and only
change the output layer. For more technical information about transfer
learning see `here <https://cs231n.github.io/transfer-learning/>`__ and
`here <https://ruder.io/transfer-learning/>`__.

In general both transfer learning methods follow the same few steps:

-  Initialize the pretrained model
-  Reshape the final layer(s) to have the same number of outputs as the
   number of classes in the new dataset
-  Define for the optimization algorithm which parameters we want to
   update during training
-  Run the training step


In [2]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from PIL import Image
#from skimage import io

PyTorch Version:  1.0.1.post2
Torchvision Version:  0.2.1


Inputs
------

Here are all of the parameters to change for the run. We will use the
*hymenoptera_data* dataset which can be downloaded
`here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`__.
This dataset contains two classes, **bees** and **ants**, and is
structured such that we can use the
`ImageFolder <https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.ImageFolder>`__
dataset, rather than writing our own custom dataset. Download the data
and set the ``data_dir`` input to the root directory of the dataset. The
``model_name`` input is the name of the model you wish to use and must
be selected from this list:

::

   [resnet, alexnet, vgg, squeezenet, densenet, inception]

The other inputs are as follows: ``num_classes`` is the number of
classes in the dataset, ``batch_size`` is the batch size used for
training and may be adjusted according to the capability of your
machine, ``num_epochs`` is the number of training epochs we want to run,
and ``feature_extract`` is a boolean that defines if we are finetuning
or feature extracting. If ``feature_extract = False``, the model is
finetuned and all model parameters are updated. If
``feature_extract = True``, only the last layer parameters are updated,
the others remain fixed.




In [62]:
# Top level data directory. Here we assume the format of the directory conforms 
#   to the ImageFolder structure
data_dir = "/home/prajna/DataScience/WiDSDatathon2019/widsdatathon2019"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "squeezenet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)

batch_size = 100

# Number of epochs to train for 
num_epochs = 10

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = True


Helper Functions
----------------

Before we write the code for adjusting the models, lets define a few
helper functions.

Model Training and Validation Code
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

The ``train_model`` function handles the training and validation of a
given model. As input, it takes a PyTorch model, a dictionary of
dataloaders, a loss function, an optimizer, a specified number of epochs
to train and validate for, and a boolean flag for when the model is an
Inception model. The *is_inception* flag is used to accomodate the
*Inception v3* model, as that architecture uses an auxiliary output and
the overall model loss respects both the auxiliary output and the final
output, as described
`here <https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958>`__.
The function trains for the specified number of epochs and after each
epoch runs a full validation step. It also keeps track of the best
performing model (in terms of validation accuracy), and at the end of
training returns the best performing model. After each epoch, the
training and validation accuracies are printed.




In [63]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            count_falsepos = 0
            count_falseneg = 0
            count_pos = 0
            count_neg = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                prednp = preds.numpy()
                datanp = labels.data.numpy()
                count_falsepos += np.sum(np.bitwise_and(prednp==1,datanp==0))
                count_falseneg += np.sum(np.bitwise_and(prednp==0,datanp==1))
                count_pos += np.sum(datanp==1)
                count_neg += np.sum(datanp==0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print('FP:{}, FN:{}, Total P:{}, Total N:{}'.format(count_falsepos,count_falseneg,count_pos,count_neg))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    

    # load best model weights
    model.load_state_dict(best_model_wts)

Set Model Parameters’ .requires_grad attribute
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

This helper function sets the ``.requires_grad`` attribute of the
parameters in the model to False when we are feature extracting. By
default, when we load a pretrained model all of the parameters have
``.requires_grad=True``, which is fine if we are training from scratch
or finetuning. However, if we are feature extracting and only want to
compute gradients for the newly initialized layer then we want all of
the other parameters to not require gradients. This will make more sense
later.




In [64]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

Initialize and Reshape the Networks
-----------------------------------

Now to the most interesting part. Here is where we handle the reshaping
of each network. Note, this is not an automatic procedure and is unique
to each model. Recall, the final layer of a CNN model, which is often
times an FC layer, has the same number of nodes as the number of output
classes in the dataset. Since all of the models have been pretrained on
Imagenet, they all have output layers of size 1000, one node for each
class. The goal here is to reshape the last layer to have the same
number of inputs as before, AND to have the same number of outputs as
the number of classes in the dataset. In the following sections we will
discuss how to alter the architecture of each model individually. But
first, there is one important detail regarding the difference between
finetuning and feature-extraction.

When feature extracting, we only want to update the parameters of the
last layer, or in other words, we only want to update the parameters for
the layer(s) we are reshaping. Therefore, we do not need to compute the
gradients of the parameters that we are not changing, so for efficiency
we set the .requires_grad attribute to False. This is important because
by default, this attribute is set to True. Then, when we initialize the
new layer and by default the new parameters have ``.requires_grad=True``
so only the new layer’s parameters will be updated. When we are
finetuning we can leave all of the .required_grad’s set to the default
of True.

Finally, notice that inception_v3 requires the input size to be
(299,299), whereas all of the other models expect (224,224).

Resnet
~~~~~~

Resnet was introduced in the paper `Deep Residual Learning for Image
Recognition <https://arxiv.org/abs/1512.03385>`__. There are several
variants of different sizes, including Resnet18, Resnet34, Resnet50,
Resnet101, and Resnet152, all of which are available from torchvision
models. Here we use Resnet18, as our dataset is small and only has two
classes. When we print the model, we see that the last layer is a fully
connected layer as shown below:

::

   (fc): Linear(in_features=512, out_features=1000, bias=True) 

Thus, we must reinitialize ``model.fc`` to be a Linear layer with 512
input features and 2 output features with:

::

   model.fc = nn.Linear(512, num_classes)

Alexnet
~~~~~~~

Alexnet was introduced in the paper `ImageNet Classification with Deep
Convolutional Neural
Networks <https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf>`__
and was the first very successful CNN on the ImageNet dataset. When we
print the model architecture, we see the model output comes from the 6th
layer of the classifier

::

   (classifier): Sequential(
       ...
       (6): Linear(in_features=4096, out_features=1000, bias=True)
    ) 

To use the model with our dataset we reinitialize this layer as

::

   model.classifier[6] = nn.Linear(4096,num_classes)

VGG
~~~

VGG was introduced in the paper `Very Deep Convolutional Networks for
Large-Scale Image Recognition <https://arxiv.org/pdf/1409.1556.pdf>`__.
Torchvision offers eight versions of VGG with various lengths and some
that have batch normalizations layers. Here we use VGG-11 with batch
normalization. The output layer is similar to Alexnet, i.e.

::

   (classifier): Sequential(
       ...
       (6): Linear(in_features=4096, out_features=1000, bias=True)
    )

Therefore, we use the same technique to modify the output layer

::

   model.classifier[6] = nn.Linear(4096,num_classes)

Squeezenet
~~~~~~~~~~

The Squeeznet architecture is described in the paper `SqueezeNet:
AlexNet-level accuracy with 50x fewer parameters and <0.5MB model
size <https://arxiv.org/abs/1602.07360>`__ and uses a different output
structure than any of the other models shown here. Torchvision has two
versions of Squeezenet, we use version 1.0. The output comes from a 1x1
convolutional layer which is the 1st layer of the classifier:

::

   (classifier): Sequential(
       (0): Dropout(p=0.5)
       (1): Conv2d(512, 1000, kernel_size=(1, 1), stride=(1, 1))
       (2): ReLU(inplace)
       (3): AvgPool2d(kernel_size=13, stride=1, padding=0)
    ) 

To modify the network, we reinitialize the Conv2d layer to have an
output feature map of depth 2 as

::

   model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))

Densenet
~~~~~~~~

Densenet was introduced in the paper `Densely Connected Convolutional
Networks <https://arxiv.org/abs/1608.06993>`__. Torchvision has four
variants of Densenet but here we only use Densenet-121. The output layer
is a linear layer with 1024 input features:

::

   (classifier): Linear(in_features=1024, out_features=1000, bias=True) 

To reshape the network, we reinitialize the classifier’s linear layer as

::

   model.classifier = nn.Linear(1024, num_classes)

Inception v3
~~~~~~~~~~~~

Finally, Inception v3 was first described in `Rethinking the Inception
Architecture for Computer
Vision <https://arxiv.org/pdf/1512.00567v1.pdf>`__. This network is
unique because it has two output layers when training. The second output
is known as an auxiliary output and is contained in the AuxLogits part
of the network. The primary output is a linear layer at the end of the
network. Note, when testing we only consider the primary output. The
auxiliary output and primary output of the loaded model are printed as:

::

   (AuxLogits): InceptionAux(
       ...
       (fc): Linear(in_features=768, out_features=1000, bias=True)
    )
    ...
   (fc): Linear(in_features=2048, out_features=1000, bias=True)

To finetune this model we must reshape both layers. This is accomplished
with the following

::

   model.AuxLogits.fc = nn.Linear(768, num_classes)
   model.fc = nn.Linear(2048, num_classes)

Notice, many of the models have similar output structures, but each must
be handled slightly differently. Also, check out the printed model
architecture of the reshaped network and make sure the number of output
features is the same as the number of classes in the dataset.




In [65]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): Fire(
      (squeeze): Conv2d(128, 32, kerne

Load Data
---------

Now that we know what the input size must be, we can initialize the data
transforms, image datasets, and the dataloaders. Notice, the models were
pretrained with the hard-coded normalization values, as described
`here <https://pytorch.org/docs/master/torchvision/models.html>`__.




In [66]:
# Data augmentation and normalization for training
# Just normalization for validation
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(input_size),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(input_size),
#         transforms.CenterCrop(input_size),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        #transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



In [67]:
# augment_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(20),
#         transforms.RandomRotation(60),
#         transforms.RandomVerticalFlip()
#     ]),
#     'val': transforms.Compose([
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(20),
#         transforms.RandomRotation(60),
#         transforms.RandomVerticalFlip()
#     ]),
# }

In [68]:
# Custom dataset inheriting torch.utils.data.Dataset to override methods
class PalmPlantationDataset(torch.utils.data.Dataset):
    """Palm Tree Plantation dataset."""
#start index and end index to track train and val split 
    def __init__(self, csv_file, start_index, end_index, root_dir, transform):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        #self.augment = augment_transforms
        
        self.start_index = start_index
        self.end_index = end_index

    def __len__(self):
        return self.end_index-self.start_index

    def __getitem__(self, idx):
        idx=idx+self.start_index
        img_name = os.path.join(self.root_dir,
                                self.labels.iloc[idx, 0])
        #image = io.imread(img_name)
        image = Image.open(img_name)
        #Extra augmenting of label=1 images since they are just 8 percent 
#       if(self.labels.iloc[idx, 1]==1):
#           image =self.augment(image)
        
        if self.transform:
            image = self.transform(image)
        label = self.labels.iloc[idx, 1]
        score = self.labels.iloc[idx, 2]
        #sample = {'image': image, 'label': label}

        return image,label

In [69]:
#Create two dataset objects , 1 training and 1 validation 
#0-13719
palms_train_dataset = PalmPlantationDataset(csv_file=data_dir+'/traininglabels.csv',
                    root_dir=data_dir+'/train_images',start_index=0,end_index=13719, transform=data_transforms['train'])
                                            #augment_transforms= augment_transforms['train'])
# Validation
#13719-15244
palms_val_dataset = PalmPlantationDataset(csv_file=data_dir+'/traininglabels.csv',
                     root_dir=data_dir+'/train_images',start_index=13719,end_index=15244, transform=data_transforms['val'])
                                          #augment_transforms= augment_transforms['val'])

In [70]:

print("Initializing Datasets and Dataloaders...")

# Create training and validation dataloaders
dataloaders_dict = {'train': torch.utils.data.DataLoader(palms_train_dataset, batch_size=batch_size, shuffle=True, num_workers=1),
 'val': torch.utils.data.DataLoader(palms_val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [71]:
print(device)

cpu


Create the Optimizer
--------------------

Now that the model structure is correct, the final step for finetuning
and feature extracting is to create an optimizer that only updates the
desired parameters. Recall that after loading the pretrained model, but
before reshaping, if ``feature_extract=True`` we manually set all of the
parameter’s ``.requires_grad`` attributes to False. Then the
reinitialized layer’s parameters have ``.requires_grad=True`` by
default. So now we know that *all parameters that have
.requires_grad=True should be optimized.* Next, we make a list of such
parameters and input this list to the SGD algorithm constructor.

To verify this, check out the printed parameters to learn. When
finetuning, this list should be long and include all of the model
parameters. However, when feature extracting this list should be short
and only include the weights and biases of the reshaped layers.




In [72]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 classifier.1.weight
	 classifier.1.bias


Run Training and Validation Step
--------------------------------

Finally, the last step is to setup the loss for the model, then run the
training and validation function for the set number of epochs. Notice,
depending on the number of epochs this step may take a while on a CPU.
Also, the default learning rate is not optimal for all of the models, so
to achieve maximum accuracy it would be necessary to tune for each model
separately.




In [73]:
# Setup the loss fxn
#criterion = nn.CrossEntropyLoss()

#Try weighted cross entropy since false negatives are high 
# 8 percent of samples are positive i.e , 942(has oilpalm)/15239(total number of images)
custom_weights = [0.08, 0.92]
class_weights = torch.FloatTensor(custom_weights)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/9
----------
train Loss: 0.1685 Acc: 0.9641
FP:402, FN:91, Total P:852, Total N:12867
val Loss: 0.1182 Acc: 0.9574
FP:62, FN:3, Total P:90, Total N:1435

Epoch 1/9
----------
train Loss: 0.1080 Acc: 0.9777
FP:248, FN:58, Total P:852, Total N:12867
val Loss: 0.0809 Acc: 0.9784
FP:28, FN:5, Total P:90, Total N:1435

Epoch 2/9
----------
train Loss: 0.0981 Acc: 0.9800
FP:222, FN:52, Total P:852, Total N:12867
val Loss: 0.0862 Acc: 0.9725
FP:38, FN:4, Total P:90, Total N:1435

Epoch 3/9
----------
train Loss: 0.0941 Acc: 0.9812
FP:205, FN:53, Total P:852, Total N:12867
val Loss: 0.0827 Acc: 0.9751
FP:35, FN:3, Total P:90, Total N:1435

Epoch 4/9
----------
train Loss: 0.0945 Acc: 0.9791
FP:232, FN:55, Total P:852, Total N:12867
val Loss: 0.1102 Acc: 0.9580
FP:64, FN:0, Total P:90, Total N:1435

Epoch 5/9
----------
train Loss: 0.0838 Acc: 0.9827
FP:194, FN:43, Total P:852, Total N:12867
val Loss: 0.0672 Acc: 0.9816
FP:24, FN:4, Total P:90, Total N:1435

Epoch 6/9
----------
train L

TypeError: cannot unpack non-iterable NoneType object

In [75]:
# Save trained model 
torch.save(model_ft, "palms_trained_model_squuezenet_weightedCE.pt")

In [76]:
# Custom dataset inheriting torch.utils.data.Dataset to override methods
class PalmTestDataset(torch.utils.data.Dataset):
    """Palm Tree Plantation dataset."""
#start index and end index to track train and val split 
    def __init__(self, root_dir, transform, list_test_filenames):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.list_test_filenames = list_test_filenames

    def __len__(self):
        return len(self.list_test_filenames)

    def __getitem__(self, idx):
        #image = io.imread(img_name)
        image = Image.open(self.root_dir+self.list_test_filenames[idx])
        if self.transform:
            image = self.transform(image)
        #sample = {'image': image, 'label': label}

        return image, self.list_test_filenames[idx]

In [77]:
#create list of test filenames by reading filenames 
holdout_list_filenames = os.listdir(data_dir+'/leaderboard_holdout_data/')
test_list_filenames = os.listdir(data_dir+'/leaderboard_test_data/')

In [78]:

#Create houldout and test dataset 
palms_holdout_dataset = PalmTestDataset(root_dir=data_dir+'/leaderboard_holdout_data/',transform=data_transforms['val'],list_test_filenames=holdout_list_filenames )

palms_test_dataset = PalmTestDataset(root_dir=data_dir+'/leaderboard_test_data/',transform=data_transforms['val'],list_test_filenames=test_list_filenames )

In [79]:
len(palms_test_dataset)

4356

In [81]:
#create dataLoader 
holdout_test_dataloader= torch.utils.data.DataLoader(palms_holdout_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
test_dataloader = torch.utils.data.DataLoader(palms_test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

In [82]:
#Test model 
def test_model(model, dataloader,is_inception=False):
    since = time.time()
    
    model.eval()   # Set model to evaluate mode

    out_all,files_all = [],[]
    # Iterate over data.
    for test_inputs,test_filenames in dataloader:
        test_inputs = test_inputs.to(device)
        #test_filenames = test_filenames.to(device)
        #print(test_inputs.shape)
        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            # Get model outputs and calculate loss
            # Special case for inception because in training it has an auxiliary output. In train
            #   mode we calculate the loss by summing the final output and the auxiliary output
            #   but in testing we only consider the final output.
            outputs = model(test_inputs)
            outputs = torch.nn.functional.softmax(outputs,dim=-1).numpy()[:,1]
            out_all.extend(outputs.tolist())
            files_all.extend(test_filenames)
            #_, preds = torch.max(outputs, 1)

    time_elapsed = time.time() - since
    print('Testing complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return out_all,files_all

In [83]:
holdout_output = test_model(model_ft,holdout_test_dataloader, is_inception=(model_name=="inception"))

Testing complete in 1m 47s


In [84]:
test_output = test_model(model_ft, test_dataloader,is_inception=(model_name=="inception") )

Testing complete in 3m 53s


In [85]:
out_all = ['has_oilpalm']+test_output[0]+holdout_output[0]
files_all = ['image_id']+test_output[1]+holdout_output[1]
np.savetxt(data_dir+"/squeezenet_output_fulldataset.csv", np.column_stack((files_all,out_all)), delimiter=",", fmt='%s')

Final Thoughts and What can be done next to improve the model
-----------------------------------

-  We can try running some of the other models such as Densenet (I have tried squeezenet here as it is fast)  and see how good the accuracy gets. Feature extracting takes less time because in the backward pass we do not have to calculate most of the gradients. 
-  Run this code with a harder dataset : Training Dataset contains just 8 percent positive labelled images > So in order to eliminate Bias , we can do data augmentation of just the positive labelled images too
- Penalize unbalanced class weight 
-  Learning Rate , other params
-  Optimization
- weighted cross entropy is applied in my solution , That can be further improved
- Look for other pretrained models that are trained on satellite images for better results 
- Change Weights
- Change to deeper network 
- More Data augmentation, I have applied very primitive data augmentation techniques 
- See if the model is overfitting or underfitting 
- Run it on full dataset with significant amount of validation data
- Keep increasing batch size until you run out of memory 
- step size can be made a parameter , change step size in factors of 10 > 0.1>0.01>0.001
- Num of Epochs : keep increasing until u see reduce in loss  : stop when loss plateaus 
